In [1]:
# Import necessary libraries
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
sys.path.append('../scripts')  # Import path

In [6]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [8]:
# Load data
filepath = r"C:\Users\HP\week 6\Credit-Scoring-Model\data\processed_data.csv"
df = pd.read_csv(filepath)


In [10]:
df.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId', 'Amount',
       'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult',
       'Total_Transaction_Amount', 'Transaction_Count',
       'ProductCategory_data_bundles', 'ProductCategory_financial_services',
       'ProductCategory_movies', 'ProductCategory_other',
       'ProductCategory_ticket', 'ProductCategory_transport',
       'ProductCategory_tv', 'ProductCategory_utility_bill',
       'ChannelId_ChannelId_2', 'ChannelId_ChannelId_3',
       'ChannelId_ChannelId_5'],
      dtype='object')

In [11]:
# Convert 'TransactionDate' to datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

In [18]:
# 1. Create Aggregate Features
def create_aggregate_features(df):
    df['TotalTransactionAmount'] = df.groupby('CustomerId')['Amount'].transform('sum')
    df['AvgTransactionAmount'] = df.groupby('CustomerId')['Amount'].transform('mean')
    df['TransactionCount'] = df.groupby('CustomerId')['Amount'].transform('count')
    df['StdTransactionAmount'] = df.groupby('CustomerId')['Amount'].transform('std')
    return df

In [19]:
# Apply aggregate features
df = create_aggregate_features(df)

In [22]:
# 2. Extract DateTime Features
def extract_datetime_features(df, datetime_column='TransactionStartTime'):
    df['TransactionHour'] = df[datetime_column].dt.hour
    df['TransactionDay'] = df[datetime_column].dt.day
    df['TransactionMonth'] = df[datetime_column].dt.month
    df['TransactionYear'] = df[datetime_column].dt.year
    return df

In [23]:
# Apply datetime feature extraction
df = extract_datetime_features(df)

In [26]:
# 3. Encode Categorical Variables
def encode_categorical(df):
    # Identify categorical columns (excluding 'Amount' and 'Value')
    categorical_cols = df.columns.difference(['Amount', 'Value'])
    
    # Apply OneHotEncoding to all categorical columns
    df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    
    return df_encoded

In [27]:
# Apply One-Hot Encoding on categorical features
df_encoded = encode_categorical(df)

MemoryError: Unable to allocate 8.45 GiB for an array with shape (95662, 94809) and data type bool

In [30]:
# Handle missing values in the dataset
def handle_missing_values(df):
    # Fill missing values for numerical columns with the median
    num_cols = ['Amount', 'Value']
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())
    
    # Fill missing values for categorical columns with the mode
    cat_cols = df.columns.difference(num_cols)
    df[cat_cols] = df[cat_cols].apply(lambda col: col.fillna(col.mode()[0]))
    
    return df

In [31]:
# Handle missing values in the dataset
df = handle_missing_values(df)

In [32]:
# 5. Normalize/Standardize Numerical Features
def scale_features(df, method='min-max'):
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    if method == 'min-max':
        scaler = StandardScaler()
        df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    return df

In [33]:
# Scale numerical features
df = scale_features(df, method='min-max')

In [34]:
# 6. Feature Engineering using WoE and IV (Stub function for now)
def calculate_woe_iv(df, target_col):
    # Placeholder for actual WoE/IV calculation logic
    print(f"Calculating WoE/IV for target: {target_col}")
    return pd.DataFrame()  # Stub DataFrame


In [35]:
# Call WoE/IV function
woe_iv_df = calculate_woe_iv(df, target_col='CreditRisk')

Calculating WoE/IV for target: CreditRisk


In [36]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,Amount,...,ChannelId_ChannelId_3,ChannelId_ChannelId_5,TotalTransactionAmount,AvgTransactionAmount,TransactionCount,StdTransactionAmount,TransactionHour,TransactionDay,TransactionMonth,TransactionYear
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,0.0,ProviderId_6,ProductId_10,-0.046371,...,True,False,0.205129,-0.067623,-0.311831,-0.168123,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,0.0,ProviderId_4,ProductId_6,-0.054643,...,False,False,0.205129,-0.067623,-0.311831,-0.168123,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,0.0,ProviderId_6,ProductId_1,-0.050426,...,True,False,0.230156,-0.072568,-0.444993,-0.202321,2,15,11,2018
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,0.0,ProviderId_1,ProductId_21,0.107717,...,True,False,0.229634,-0.008155,-0.404020,-0.009328,3,15,11,2018
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,0.0,ProviderId_4,ProductId_6,-0.059704,...,False,False,0.229634,-0.008155,-0.404020,-0.009328,3,15,11,2018


In [39]:
# Function to calculate risk score
def calculate_risk(df):
    # Define your risk calculation logic here
    # This is a simple example: adjust as needed based on your business rules
    df['Risk'] = 0  # Initialize Risk column

    # Example conditions for risk calculation
    df.loc[df['Amount'] > 1000, 'Risk'] = 2  # High risk for high amounts
    df.loc[(df['Amount'] <= 1000) & (df['Value'] > 100), 'Risk'] = 1  # Medium risk
    df.loc[(df['Amount'] <= 1000) & (df['Value'] <= 100), 'Risk'] = 0  # Low risk

    return df


In [40]:
# Calculate risk score
df= calculate_risk(df)

In [41]:
# Save processed data
df.to_csv("C:\\Users\\HP\\week 6\\Credit-Scoring-Model\\data\\processed_data1.csv", index=False)

In [42]:
# Display the first few rows of the processed dataset
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,Amount,...,ChannelId_ChannelId_5,TotalTransactionAmount,AvgTransactionAmount,TransactionCount,StdTransactionAmount,TransactionHour,TransactionDay,TransactionMonth,TransactionYear,Risk
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,0.0,ProviderId_6,ProductId_10,-0.046371,...,False,0.205129,-0.067623,-0.311831,-0.168123,2,15,11,2018,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,0.0,ProviderId_4,ProductId_6,-0.054643,...,False,0.205129,-0.067623,-0.311831,-0.168123,2,15,11,2018,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,0.0,ProviderId_6,ProductId_1,-0.050426,...,False,0.230156,-0.072568,-0.444993,-0.202321,2,15,11,2018,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,0.0,ProviderId_1,ProductId_21,0.107717,...,False,0.229634,-0.008155,-0.404020,-0.009328,3,15,11,2018,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,0.0,ProviderId_4,ProductId_6,-0.059704,...,False,0.229634,-0.008155,-0.404020,-0.009328,3,15,11,2018,0
